#### Import libraries

In [5]:
import pandas as pd, numpy as np, re, textwrap, string
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from pathlib import Path


#### Setting the text column characters width to 200

In [ ]:
pd.set_option("display.max_colwidth", 200)

#### Load Data

In [6]:
data = Path("../data")

In [7]:
train_path = data/"train.csv"
test_path = data/"test.csv"
sample_submission_path = data/"sample_submission.csv"

In [8]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)

#### Standardize column names in ***'train'*** dataset

In [9]:
train.columns = [c.strip().lower() for c in train.columns]
target = "rule_violation"
text_cols = ["body", "rule", "positive_example_1", "positive_example_2", "negative_example_1", "negative_example_2"]
cat_cols = ["subreddit"]
id_cols = ["row_id"]

#### Exploring the ***train*** dataset

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


In [11]:
train.sample(5, random_state=42)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
1356,1356,All people visit your sister therein such a tits! http://codimenu.com/ac/16php,"No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",funny,49 http://MySexFind.com - find girl for sex now 49!,FindSexToday . com - free girls for sex worldwide 3oBKmqU6XiS14!,SD Stream [Spanish Stream](http://www.expresssports.ml/p/adsbygoogle-window_146.html) \nMobile : Not Compatible,Here's the link. \n\nhttp://www.powerhousetradingsystem.com/trading.html,1
984,984,Try and see if someone at www.siddhantayoga.com can help you ...,"No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",SuicideWatch,Ray-Ban Sunglasses Special Sale at $24.99 Shop Now >>> http://www.rbsclassic.com,Help my mans get to 500 dollars\nhttp://referdone.com/?share=10399,If anyone is interested in donating 5 or 10 bucks https://timcanova.com/,"Same shit happened to me man. EXACT same thing, so it's probably AmWay. They got that procedure down pat, probably because it works so well. At the end, I tried to reason with the guy because h...",0
859,859,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/STREAM:\n\nhttp://music.theblacksmithed.com/download/birds/","No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",movies,My pecker tried best holes at this point now you win this prize http://hothor.se/1xqhk,Trying to raise some funds on PayPal for a boob job any donations appreciated 1-100 I'm up for anything have loads of pictures and videos to share also just about to shower so can take more 😘😘 I'm...,How to Record Android or IPhone Screen Without Rooted Devices . Full Tutorial Watch here https://m.youtube.com/watch?v=FyD2MYbSnKE,SD Stream Eng - [Chelsea TV USA](http://soccerstreamsarena.com/chelsea-u23-vs-tottenham-u23-live-epl2-09-sep-2016/) \n\nMobile : Compatible With Android Puffin Browser,0
1983,1983,WWW.CSGOPOT.COM USE PROMO: CSGOPOTPRO FOR FREE DIAMONDS WWW.CSGOPOT.COM,"No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",GlobalOffensive,**Dortmund vs Real Madrid Live Stream**\n\n**<<< http://www.dortmund-vs-realmadrid-live.cf >>>**\n\n**HD Quality Streams**\n\n**<<< http://www.dortmund-vs-realmadrid-live.cf >>>**\n\n**HD SOUND Qu...,Selling Tyrande code for 3€ to paypal. PM if interested.\n\nProof of satisfied Tyrande owners: http://imgur.com/a/jydZ7 http://imgur.com/a/jIQGQ\n,"Ive got an Bayonet | Damascus Steel ft, if your're interested \n\nhttps://steamcommunity.com/tradeoffer/new/?partner=215103279&token=fCan2TeW","Shameless plug for our Facebook page, all home made memes https://www.facebook.com/IMakeBleachAndDrinkMemes",1
1293,1293,#stop deleting comments. What you did was fraud.\nhttps://sli.mg/3GGiCy,No legal advice: Do not offer or request legal advice.,The_Donald,Well first you have to figure out if there is even a way that you can do such an agreement and you have to talk to a lawyer about that.,"Maybe true, but that's very short-sighted. The lawsuits are already in the hundreds though on this. He's going to be getting sued six ways to sunset for his entire term. The Constitution offers ve...","> Yeah she initiated it **when I was sleeping**\n\nThat's certainly the unambiguous part of the story. Sexual assault, no gray area.\n\n","If it's *only* killing sick, elderly people - isn't it kind of... alright?",0


In [12]:
train[target].value_counts(dropna=False).to_frame("count").assign(pct=lambda t: t["count"]/len(train))

,count,pct
rule_violation,,
1,1031,0.508132
0,998,0.491868


In [13]:
train.isna().mean().sort_values(ascending=False)

row_id                0.0
body                  0.0
rule                  0.0
subreddit             0.0
positive_example_1    0.0
positive_example_2    0.0
negative_example_1    0.0
negative_example_2    0.0
rule_violation        0.0
dtype: float64

#### Let's count how many rows are exact duplicates of earlier rows in all columns.

In [18]:
train.duplicated().sum()

np.int64(0)

#### Let's counts how many times the same row ID appears more than once

In [19]:
train[id_cols[0]].duplicated().sum()

np.int64(0)

#### Checking Target leakage in the train dataset

##### Hard leakage

In [ ]:
matches = 0

for col in ["positive_example_1", "positive_example_2", "negative_example_1", "negative_example_2"]:
    matches += train.merge(
        train[["row_id", col]].rename(columns = {col:"x"}),
        left_on = "body",
        right_on = "x",
        how = "inner"
    ).shape[0]

matches

12902

##### Soft Leakage

In [ ]:
# top_ngrams() is defined to find the most frequent n-grams
def top_ngrams(texts, ngram_range = (2,2), topk=2000): 
    v = CountVectorizer(ngram_range=ngram_range, min_df=2) # min_df=2 means it only include phrases that appear in at least 2 documents
    X = v.fit_transform(texts)
    freqs = np.asarray(X.sum(0)).ravel()
    order = freqs.argsort()[::-1][:topk]
    vocab = np.array(sorted(v.vocabulary_.items(), key=lambda kv:kv[1]))[:, 0]
    return set(vocab[order])

body_bi = top_ngrams(train["body"].fillna(""),(2,2), 3000)
ex_all = pd.Series(train[text_cols[1:]].fillna("").agg(" ".join, axis=1))
ex_bi = top_ngrams(ex_all, (2,2), 3000)
len(body_bi & ex_bi )


1776

# Checking out for Missing values & basic cleaning

In [ ]:
# How often each text field is empty/whitespace

def empty_rate(s): 
    return s.fillna("").str.strip().eq("").mean()
pd.Series({c: empty_rate(train[c]) for c in text_cols + cat_cols})

body                  0.0
rule                  0.0
positive_example_1    0.0
positive_example_2    0.0
negative_example_1    0.0
negative_example_2    0.0
subreddit             0.0
dtype: float64

In [28]:
# normalize() creates Simple normalized versions for analysis

def normalize(s):
    s = s.fillna("").str.replace(r"\s+", " ", regex=True).str.strip()                           
                    # Replace NaN/None with empty string 
                    # Replace multiple whitespace (spaces, tabs, newlines) with a single space
                    # Remove leading and trailing spaces
    return s

for c in text_cols:
    train[c+"_clean"] = normalize(train[c])

train["subreddit"] = train["subreddit"].astype("category")

#### Exploring Columns

##### *I] subreddit*

In [29]:
sub_counts = train["subreddit"].value_counts().to_frame("count").assign(pct=lambda t: t["count"]/len(train))
sub_counts.head(20)


,count,pct
subreddit,,
legaladvice,213,0.104978
AskReddit,152,0.074914
soccerstreams,139,0.068507
personalfinance,125,0.061607
relationships,106,0.052242
The_Donald,94,0.046328
TwoXChromosomes,87,0.042878
news,65,0.032035
movies,56,0.027600


In [ ]:
# Calculating how often rule violations happen in each subreddit - only for subreddits that have enough data to be meaningful

tmp = (train.groupby("subreddit")[target]
         .agg(["count","mean"])
         .rename(columns={"mean":"violation_rate"})
         .query("count >= 30")
         .sort_values("violation_rate", ascending=False))

tmp.head(20)

/var/folders/p0/bs5mtjhd6016xb_dz27dp3t00000gn/T/ipykernel_36283/280065292.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tmp = (train.groupby("subreddit")[target]


,count,violation_rate
subreddit,,
sex,42,0.809524
legaladvice,213,0.788732
personalfinance,125,0.696000
relationships,106,0.613208
politics,49,0.591837
news,65,0.584615
pics,30,0.566667
The_Donald,94,0.553191
AskReddit,152,0.480263


##### *II] subreddit*

In [ ]:
rule_counts = (train["rule"].fillna("NA").value_counts()
               .to_frame("count")
               .assign(pct=lambda t: t["count"]/len(train)))

rule_counts.head(20)




,count,pct
rule,,
No legal advice: Do not offer or request legal advice.,1017,0.501232
"No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",1012,0.498768


In [34]:
# Violation rate per rule (for rules with support)
rule_stats = (train.assign(rule=train["rule"].fillna("NA"))
                .groupby("rule")[target]
                .agg(["count","mean"])
                .rename(columns={"mean":"violation_rate"})
                .query("count >= 20")
                .sort_values("violation_rate", ascending=False))
rule_stats.head(20)

,count,violation_rate
rule,,
No legal advice: Do not offer or request legal advice.,1017,0.583088
"No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.",1012,0.432806


In [ ]:
/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv